In [3]:
import pandas as pd
df = pd.read_csv('C:/Users/tomas/OneDrive/Escritorio/ORGA DE DATOS/business.csv', usecols = ["business_id","review_count","name"])
df2= pd.read_csv('C:/Users/tomas/OneDrive/Escritorio/ORGA DE DATOS/review.csv', usecols = ["business_id","text"])

In [4]:
nuevo = pd.merge(df2, df,on = "business_id")
nuevo = nuevo[nuevo.review_count > 10]

In [5]:
group = nuevo.groupby(["business_id","name","review_count"])[["text"]].agg(list)

In [6]:
nuevo_df = pd.DataFrame(group)
nuevo_df = nuevo_df.reset_index()

In [7]:
sample = nuevo_df.sample(frac=0.005, random_state=1)

In [8]:
X_train = []
for texts in sample.text:
    texto_concat = ""
    for text in texts:
        texto_concat = texto_concat + text
    X_train.append(texto_concat)    

In [16]:
X_test = X_train

In [20]:
import zstandard
import gzip

def compress_size(bytefile):
    return len(zstandard.compress((bytefile.encode())))


def ncd(st1,st2):
    len1 = compress_size(st1)
    len2 = compress_size(st2)
    concat_len =  compress_size(st1+st2)
    return (concat_len - min(len1,len2))/max(len1,len2)


In [21]:
from tqdm import tqdm
import numpy as np
distances = []

for testfile in tqdm(X_test):
    vect = []
    for file in X_train:
        if(file != testfile):
            vect.append(ncd(file, testfile))
        else:
            vect.append(1)
    distances.append(vect)

100%|████████████████████████████████████████████████████████████████████████████████| 319/319 [01:09<00:00,  4.56it/s]


In [22]:
distances = np.array(distances)
rows, cols = np.where(distances == np.min(distances))
negocio_uno = nuevo_df.name[rows[0]]
negocio_dos = nuevo_df.name[cols[0]]
texto_uno = nuevo_df.text[rows[0]]
texto_dos = nuevo_df.text[cols[0]]
distancia = np.min(distances)

In [23]:
print("Los son negocios mas parecidos son",negocio_uno,"y",negocio_dos)
print("cuyos textos son:")
print("Texto UNO", texto_uno)


Los son negocios mas parecidos son Piscobar y La Fava Restaurant and Bar
cuyos textos son:
Texto UNO ["One of my favorite places - great coffee, food, and bar.  The staff here is the best - always pleasant and they know their customers.  If you work in the CBD, I recommend this as your morning coffee stop.  The lunch menu is excellent and the prices are very reasonable.  After work is the perfect time to hit the rooftop bar - happy hour and daily drink specials abound.  I've been coming here since they opened in April - highly recommend it!", 'Great location and the rooftop lounge area of the hotel is beautiful, but the service killed the experience. The bartender took a while to make two drinks, and even though she was lacking a key ingredient to make them, she handed us the drinks and explained she "made it work". The drinks were bitter and very hard to drink. She then charged us a different price than what was in their menu, and acted as if she was doing us a favor by giving us corr

In [24]:
print("Texto DOS", texto_dos)


Texto DOS ["Best Italian I've had in this area. It's my new go to place. I really liked the seafood dishes and the staff is very nice. Also it's open on Mondays at lunch which seems rare in this area.", 'We had a wonderful meal no one was disappointed. I had the gnocchi with sausage and mushrooms itvwas fabulous. Our waiter Jabar was superb , thank you for your wonderful service. We will be back !', 'We will be Back!\nHad dinner with 3 other couples. Table-side\nBronzino, beautiful pasta dishes, terrific apps and drinks. So nice inside. Welcome\nTo the neighborhood ! Heading back tonight', 'Amazing first impression! My girlfriend and I tried out La fava for the first time last night and we were very impressed. This is a restaurant glen mills needed. Specialty cocktails were some of the best we ever had. High quality ingredients/Alcohol for a fairly good price. Wine list is a little limited but it just opened so expect it to expand. Most importantly the staff/food stole the show. Must t

In [25]:
print("Su distancia es:",distancia)

Su distancia es: 0.8698159237081393
